# Audio Data Exploration

This notebook demonstrates how to explore and analyze audio data using the music preprocessing pipeline.

## Topics Covered:
- Loading and visualizing audio files
- Basic audio statistics and properties
- Audio quality validation
- Preprocessing audio for analysis

## Setup and Imports

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add src directory to path
sys.path.append('../src')

# Import our preprocessing modules
from audio_loader import AudioLoader
from utils.audio_utils import validate_audio_quality, preprocess_audio_standard
from utils.visualization import plot_waveform, create_audio_analysis_report

print("Modules imported successfully!")

## Loading Audio Files

Let's start by loading an audio file and examining its properties.

In [ ]:
# Initialize audio loader
loader = AudioLoader(target_sr=22050, mono=True, normalize=True)

# Example: Load an audio file (replace with your audio file path)
# audio_file = "../data/examples/sample_song.mp3"
# 
# For demonstration, we'll create a synthetic audio signal
# In practice, you would load a real audio file

# Create a synthetic audio signal for demonstration
duration = 5.0  # seconds
sample_rate = 22050
t = np.linspace(0, duration, int(duration * sample_rate))

# Create a complex signal with multiple frequencies and some noise
audio_data = (
    0.5 * np.sin(2 * np.pi * 440 * t) +  # A4 note
    0.3 * np.sin(2 * np.pi * 880 * t) +  # A5 note
    0.2 * np.sin(2 * np.pi * 220 * t) +  # A3 note
    0.1 * np.random.randn(len(t))        # Some noise
)

print(f"Audio loaded:")
print(f"  Duration: {len(audio_data) / sample_rate:.2f} seconds")
print(f"  Sample rate: {sample_rate} Hz")
print(f"  Number of samples: {len(audio_data)}")
print(f"  Data type: {audio_data.dtype}")
print(f"  Min amplitude: {np.min(audio_data):.4f}")
print(f"  Max amplitude: {np.max(audio_data):.4f}")

## Audio Visualization

Let's visualize the audio waveform to understand its characteristics.

In [ ]:
# Plot the full waveform
plot_waveform(audio_data, sample_rate, title="Complete Audio Waveform")

# Plot a zoomed-in section to see more detail
start_sample = int(1.0 * sample_rate)  # Start at 1 second
end_sample = int(2.0 * sample_rate)    # End at 2 seconds
audio_segment = audio_data[start_sample:end_sample]

plot_waveform(audio_segment, sample_rate, title="Audio Waveform (1-2 seconds)")

## Audio Quality Validation

Before processing, let's validate the audio quality to identify potential issues.

In [ ]:
# Validate audio quality
quality_results = validate_audio_quality(audio_data, sample_rate)

print("Audio Quality Assessment:")
print(f"  Overall Quality Score: {quality_results['quality_score']:.2f}/1.0")
print(f"  Duration OK: {quality_results['duration_ok']} ({quality_results['duration']:.2f}s)")
print(f"  Clipping OK: {quality_results['clipping_ok']} ({quality_results['clipping_ratio']:.4f} ratio)")
print(f"  Silence OK: {quality_results['silence_ok']} ({quality_results['silence_ratio']:.4f} ratio)")
print(f"  Dynamic Range OK: {quality_results['dynamic_range_ok']} ({quality_results['dynamic_range']:.4f})")
print(f"  DC Bias OK: {quality_results['dc_bias_ok']} ({quality_results['dc_bias']:.4f})")
print(f"  Overall Assessment: {'PASS' if quality_results['overall_ok'] else 'FAIL'}")

## Audio Statistics

Let's compute some basic statistics about our audio signal.

In [ ]:
# Compute basic statistics
print("Audio Statistics:")
print(f"  Mean: {np.mean(audio_data):.6f}")
print(f"  Standard Deviation: {np.std(audio_data):.6f}")
print(f"  RMS Energy: {np.sqrt(np.mean(audio_data**2)):.6f}")
print(f"  Peak Amplitude: {np.max(np.abs(audio_data)):.6f}")
print(f"  Zero Crossings: {np.sum(np.diff(np.sign(audio_data)) != 0)}")

# Compute spectral statistics using librosa
import librosa

# Spectral centroid (brightness)
spectral_centroids = librosa.feature.spectral_centroid(y=audio_data, sr=sample_rate)[0]
print(f"  Spectral Centroid (mean): {np.mean(spectral_centroids):.2f} Hz")

# Spectral rolloff
spectral_rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sample_rate)[0]
print(f"  Spectral Rolloff (mean): {np.mean(spectral_rolloff):.2f} Hz")

# Zero crossing rate
zcr = librosa.feature.zero_crossing_rate(audio_data)[0]
print(f"  Zero Crossing Rate (mean): {np.mean(zcr):.6f}")

## Audio Preprocessing

Let's apply standard preprocessing steps to prepare the audio for feature extraction.

In [ ]:
# Apply standard preprocessing
processed_audio, processed_sr = preprocess_audio_standard(
    audio_data, 
    sample_rate,
    target_sr=22050,
    normalize_method="peak",
    trim_silence_flag=True
)

print("Preprocessing Results:")
print(f"  Original length: {len(audio_data)} samples ({len(audio_data)/sample_rate:.2f}s)")
print(f"  Processed length: {len(processed_audio)} samples ({len(processed_audio)/processed_sr:.2f}s)")
print(f"  Sample rate: {processed_sr} Hz")

# Compare original vs processed
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Original audio
time_orig = np.linspace(0, len(audio_data) / sample_rate, len(audio_data))
ax1.plot(time_orig, audio_data, alpha=0.8)
ax1.set_title('Original Audio')
ax1.set_ylabel('Amplitude')
ax1.grid(True, alpha=0.3)

# Processed audio
time_proc = np.linspace(0, len(processed_audio) / processed_sr, len(processed_audio))
ax2.plot(time_proc, processed_audio, alpha=0.8, color='orange')
ax2.set_title('Processed Audio')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Amplitude')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Frequency Domain Analysis

Let's examine the frequency content of our audio signal.

In [ ]:
# Compute FFT for frequency analysis
fft = np.fft.fft(processed_audio)
fft_magnitude = np.abs(fft)
fft_freq = np.fft.fftfreq(len(processed_audio), 1/processed_sr)

# Only plot positive frequencies
positive_freq_idx = fft_freq >= 0
fft_freq_positive = fft_freq[positive_freq_idx]
fft_magnitude_positive = fft_magnitude[positive_freq_idx]

# Plot frequency spectrum
plt.figure(figsize=(12, 6))
plt.plot(fft_freq_positive, fft_magnitude_positive)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Frequency Spectrum')
plt.xlim(0, 2000)  # Focus on lower frequencies
plt.grid(True, alpha=0.3)

# Mark the fundamental frequencies we used
plt.axvline(x=220, color='red', linestyle='--', alpha=0.7, label='A3 (220 Hz)')
plt.axvline(x=440, color='green', linestyle='--', alpha=0.7, label='A4 (440 Hz)')
plt.axvline(x=880, color='blue', linestyle='--', alpha=0.7, label='A5 (880 Hz)')
plt.legend()

plt.tight_layout()
plt.show()

# Find dominant frequencies
peak_indices = np.argsort(fft_magnitude_positive)[-10:]  # Top 10 peaks
dominant_freqs = fft_freq_positive[peak_indices]

print("Top 10 Dominant Frequencies:")
for i, freq in enumerate(sorted(dominant_freqs[dominant_freqs > 0])[:10]):
    print(f"  {i+1}. {freq:.1f} Hz")

## Working with Real Audio Files

Here's how you would work with real audio files from your dataset.

In [ ]:
# Example of working with real audio files
# Uncomment and modify the following code to work with your audio files

# # Define path to your audio files
# audio_dir = Path("../data/raw_audio")
# 
# # Get list of audio files
# if audio_dir.exists():
#     audio_files = list(audio_dir.glob("*.mp3")) + list(audio_dir.glob("*.wav"))
#     print(f"Found {len(audio_files)} audio files")
#     
#     # Load and analyze the first file
#     if audio_files:
#         first_file = audio_files[0]
#         print(f"Analyzing: {first_file.name}")
#         
#         # Load audio
#         audio_data, sr = loader.load_audio(first_file)
#         
#         # Get file info
#         file_info = loader.get_audio_info(first_file)
#         print(f"Duration: {file_info['duration']:.2f} seconds")
#         print(f"Sample rate: {file_info['sample_rate']} Hz")
#         print(f"Channels: {file_info['channels']}")
#         
#         # Validate quality
#         quality = validate_audio_quality(audio_data, sr)
#         print(f"Quality score: {quality['quality_score']:.2f}")
#         
#         # Create visualization
#         plot_waveform(audio_data, sr, title=f"Waveform: {first_file.name}")
# else:
#     print("Audio directory not found. Create ../data/raw_audio/ and add some audio files.")

print("Ready to analyze real audio files!")
print("To use this notebook with your audio files:")
print("1. Create a 'data/raw_audio' directory")
print("2. Add some audio files (.mp3, .wav, etc.)")
print("3. Uncomment and run the code above")

## Summary

In this notebook, we've explored:
- How to load and validate audio files
- Basic audio statistics and properties
- Audio preprocessing techniques
- Frequency domain analysis
- Visualization of audio data

This foundational understanding of audio data will help you in the next steps of feature extraction and model preparation.

## Next Steps
- Check out `02_feature_analysis.ipynb` for feature extraction
- Explore `03_spectrogram_visualization.ipynb` for spectrogram analysis
- Start building your dataset using the batch processing tools